# 机器学习大作业


## 数据预处理

### 1 导入数据

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE

df = pd.read_csv('data.csv')
df = df.applymap(lambda x: np.nan if x==0 else x)
grouped = df.groupby('诊断名称')
sub_dfs = {name: group for name, group in grouped}

sub_dfs['肝硬化'].interpolate(method='linear', inplace=True)
sub_dfs['肝炎'].interpolate(method='linear', inplace=True)
sub_dfs['肝癌'].interpolate(method='linear', inplace=True)

df = pd.concat([sub_dfs['肝硬化'], sub_dfs['肝癌'], sub_dfs['肝炎']])
df = df.dropna()
df_dict = {'data': df.drop(['ID','诊断名称'], axis=1), 'target': df['诊断名称']}
X = df_dict['data'].to_numpy()
y = df_dict['target'].to_numpy()
feature_names = df_dict['data'].columns
print(feature_names)
# 标准化数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)


Index(['白蛋白(ALB)-静脉血', '白蛋白/球蛋白比值(ALB/GLO)-静脉血', '球蛋白(GLO)-静脉血', '总蛋白(TP)-静脉血',
       '丙氨酸氨基转移酶(ALT)-静脉血', '天门冬氨酸氨基转移酶/丙氨酸氨基转移酶比值(AST/ALT)-静脉血',
       '天门冬氨酸氨基转移酶(AST)-静脉血', '间接胆红素(IBIL)-静脉血', '总胆红素(TBIL)-静脉血',
       '碱性磷酸酶(ALP)-静脉血', '胆碱酯酶(ChE)-静脉血', '白细胞计数(WBC#)-静脉血',
       '单核细胞百分比(Mono%)-静脉血', '单核细胞计数(Mono#)-静脉血', '红细胞计数(RBC#)-静脉血',
       '红细胞分布广度-SD-静脉血', '红细胞比容(Hct)-静脉血', '淋巴细胞计数(Lymph#)-静脉血',
       '淋巴细胞百分比(Lymph%)-静脉血', '平均红细胞体积(MCV)-静脉血', '平均红细胞血红蛋白含量(MCH)-静脉血',
       '平均红细胞血红蛋白浓度(MCHC)-静脉血', '嗜酸性粒细胞计数(Eos#)-静脉血', '嗜碱性粒细胞计数(Baso#)-静脉血',
       '嗜酸性粒细胞百分比(Eos%)-静脉血', '嗜碱性粒细胞百分比(Baso%)-静脉血', '血红蛋白(Hb)-静脉血',
       '血小板比容-静脉血', '血小板计数(PLT#)-静脉血', '平均血小板体积(MPV)-静脉血', '血小板分布宽度(PDW)-静脉血',
       '中性粒细胞计数(Neut#)-静脉血', '中性粒细胞百分比(Neut%)-静脉血', '阴离子间隙(AG)-静脉血',
       '钾离子(K+)-静脉血', '钠离子(Na+)-静脉血', '氯离子(Cl-)-静脉血', '总钙(Ca)-静脉血',
       '镁离子(Mg2+)-静脉血', 'γ-谷氨酰基转移酶(GGT)-静脉血'],
      dtype='object')


### 2 过滤法  Relief-F 方法

#### 基于互信息

In [28]:
from sklearn.feature_selection import mutual_info_classif

mi = mutual_info_classif(X, y)

# 获取互信息值的排序索引
sorted_indices = np.argsort(-mi)

# 获取排序后的互信息值
sorted_mi = mi[sorted_indices]

n_repeats = 10
feature_names = df_dict['data'].columns.tolist()

# 初始化一个列表，用于存储每个特征的累计互信息值
cumulative_mi = np.zeros(len(feature_names))

# 多次计算互信息并累加
for _ in range(n_repeats):
    mi = mutual_info_classif(X, y)
    cumulative_mi += mi
    

# 获取排序后的互信息值索引
sorted_indices = np.argsort(-cumulative_mi)

# 打印排序后的互信息值及其原始位置
i = 0
print("Cumulative Mutual Information for each feature:")
for idx in sorted_indices:
    print(f"Feature {feature_names[idx]}: Cumulative Mutual Information = {cumulative_mi[idx]}")
    i = i + 1
    if i == 9:
        break



Cumulative Mutual Information for each feature:
Feature 胆碱酯酶(ChE)-静脉血: Cumulative Mutual Information = 1.105623789065392
Feature 血小板计数(PLT#)-静脉血: Cumulative Mutual Information = 1.0248667555525202
Feature 血小板比容-静脉血: Cumulative Mutual Information = 0.9957505652859344
Feature 红细胞分布广度-SD-静脉血: Cumulative Mutual Information = 0.7317568241251706
Feature 间接胆红素(IBIL)-静脉血: Cumulative Mutual Information = 0.6904131351449017
Feature 天门冬氨酸氨基转移酶/丙氨酸氨基转移酶比值(AST/ALT)-静脉血: Cumulative Mutual Information = 0.6519467501768361
Feature 总胆红素(TBIL)-静脉血: Cumulative Mutual Information = 0.6492695866038196
Feature 白蛋白(ALB)-静脉血: Cumulative Mutual Information = 0.6198432270655982
Feature 红细胞计数(RBC#)-静脉血: Cumulative Mutual Information = 0.5711521756717852


### 3 包裹法

In [8]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

# 获取特征重要性
importances = model.feature_importances_

# 将特征重要性与特征名称对应起来
feature_importances = pd.DataFrame({'feature': df.drop(['ID', '诊断名称'], axis=1).columns, 'importance': importances})

# 按重要性排序
feature_importances = feature_importances.sort_values(by='importance', ascending=False)




# 选择最重要的N个特征，这里N需要根据实际情况来定
N = 40  
num_repeat = 3
acc_list =[[0]*len(feature_importances) for i in range(num_repeat)]
print('***************')

for j in range(num_repeat):
    print('-------------------------------------------')
    for i in range(40):
        selected_features = feature_importances.head(N-i)['feature']
    
        # 使用选定的特征重新训练模型
        X_train_selected = X_train[:, selected_features.index]
        X_test_selected = X_test[:, selected_features.index]
    
        model_selected = RandomForestClassifier(random_state=42)
        model_selected.fit(X_train_selected, y_train)
        y_pred_selected = model_selected.predict(X_test_selected)
    
        # 计算新模型的准确率
        acc_selected = accuracy_score(y_test, y_pred_selected)
        print(f'{N-i} 使用选定特征的准确率：{acc_selected}' )
        acc_list[j].append(acc_selected)
    print('-------------------------------------------')



***************
-------------------------------------------
40 使用选定特征的准确率：0.6719457013574661
39 使用选定特征的准确率：0.6719457013574661
38 使用选定特征的准确率：0.6561085972850679
37 使用选定特征的准确率：0.6425339366515838
36 使用选定特征的准确率：0.665158371040724
35 使用选定特征的准确率：0.6583710407239819
34 使用选定特征的准确率：0.6515837104072398
33 使用选定特征的准确率：0.669683257918552
32 使用选定特征的准确率：0.6538461538461539
31 使用选定特征的准确率：0.6764705882352942
30 使用选定特征的准确率：0.6628959276018099
29 使用选定特征的准确率：0.6628959276018099
28 使用选定特征的准确率：0.6583710407239819
27 使用选定特征的准确率：0.6628959276018099
26 使用选定特征的准确率：0.6515837104072398
25 使用选定特征的准确率：0.6606334841628959
24 使用选定特征的准确率：0.6628959276018099
23 使用选定特征的准确率：0.6493212669683258
22 使用选定特征的准确率：0.6628959276018099
21 使用选定特征的准确率：0.6561085972850679
20 使用选定特征的准确率：0.6606334841628959
19 使用选定特征的准确率：0.6538461538461539
18 使用选定特征的准确率：0.6447963800904978
17 使用选定特征的准确率：0.6515837104072398
16 使用选定特征的准确率：0.6470588235294118
15 使用选定特征的准确率：0.6470588235294118
14 使用选定特征的准确率：0.6515837104072398
13 使用选定特征的准确率：0.6447963800904978
12 使用选定特征的准确率：0.64

In [ ]:
cumulative_sum_columns = np.cumsum(X, axis=0).sum(axis=0)
acc_max = max(cumulative_sum_columns)
print(acc_max)
max_index = cumulative_sum_columns.index(acc_max)
print(max_index+1)
print(feature_importances[:max_index+1])

### 4 嵌入法 

In [28]:

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier as RFC

# 实例化随机森林
RFC_ = RFC(n_estimators=10, random_state=0)
# 使用SelectFromModel进行特征选择
selector =  SelectFromModel(RFC_, threshold=0.035)
X_embedded = selector.fit_transform(X, y)

# 获取被选择特征的布尔掩码
selected_features_mask = selector.get_support()

# 根据掩码获取对应的特征名称
selected_feature_names = [feature_names[i] for i in range(len(feature_names)) if selected_features_mask[i]]

# 输出被选择的特征名称
print("Selected features:")
print(selected_feature_names)

# 如果需要，也可以输出被选择的特征数量
print("Number of selected features:", sum(selected_features_mask))

Selected features:
['总胆红素(TBIL)-静脉血', '胆碱酯酶(ChE)-静脉血', '红细胞分布广度-SD-静脉血', '血小板比容-静脉血', '血小板计数(PLT#)-静脉血']
Number of selected features: 5


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 数据准备
features = ['胆碱酯酶(ChE)-静脉血', '血小板计数(PLT#)-静脉血', '血小板比容-静脉血', '天门冬氨酸氨基转移酶/丙氨酸氨基转移酶比值(AST/ALT)-静脉血', '总胆红素(TBIL)-静脉血']
filter_importance = [0.069706, 0.057907, 0.046705, 0.040405, 0.037988]
mutual_info = [1.105623789065392, 1.0248667555525202, 0.9957505652859344, None, 0.6492695866038196]
embedded_importance = [0.037988, 0.069706, 0.7317568241251706, 0.046705, 0.057907]

# 创建一个新的数据框架用于绘图
df = pd.DataFrame({
    'Feature': features,
    'Filter Importance': filter_importance,
    'Mutual Info': mutual_info,
    'Embedded Importance': embedded_importance
}).set_index('Feature')

# 绘制折线图
plt.figure(figsize=(12, 6))
for method in df.columns:
    plt.plot(df.index, df[method], marker='o', label=method)

plt.title('特征选择方法重要性对比', fontsize=14)
plt.xlabel('特征', fontsize=12)
plt.ylabel('重要性', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
